In [1]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm, trange
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from torch.utils.data import Dataset, DataLoader

In [3]:
## create dataset
class DroughtDataset(Dataset):
    """ Drought dataset."""

    def __init__(self, np_array_x, np_array_y):
        """
        Args:
            np_array_x (string): Path to the npy file with annotations.
            np_array_y (string): Path to the npy file with annotations.
        """
        self.X = np.load(np_array_x)
        self.Y = np.load(np_array_y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx][0]

In [26]:
drought_dataset = DroughtDataset('X_v2.npy', 'Y_v2.npy')

for i in range(len(drought_dataset)):
    sample = drought_dataset[i]
    # sample[0].shape = (6,4)

In [5]:
## call dataloader
dataloader = DataLoader(drought_dataset, batch_size=2, shuffle=True, num_workers=0)
# batch,seq,input_var

In [15]:
## create LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=4, hidden_layer_size=32, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size,batch_first=True)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, hidden_cell = self.lstm(input_seq)#, self.hidden_cell.cuda())
        predictions = self.linear(lstm_out[:, -1])
        return predictions.squeeze(1)

In [29]:
## cross entropy loss and optimizer
model = LSTM()
model = model
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [30]:
## training the model
epochs = 150

for i in trange(epochs):
    sum_loss = 0
    for batch in dataloader:
        seq, labels = batch
        seq, labels = seq.float(), labels.float()
        optimizer.zero_grad()
        #model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                torch.zeros(1, 1, model.hidden_layer_size))
        y_pred = model(seq)
        single_loss = loss_function(y_pred, labels)
        
        single_loss.backward()
        optimizer.step()
        sum_loss += single_loss
        
    if i%5 == 1:
        print(f'epoch: {i:3} loss: {sum_loss.cpu().item()/len(dataloader):10.8f}')

print(f'epoch: {i:3} loss: {single_loss.cpu().item():10.10f}')

  0%|          | 0/150 [00:00<?, ?it/s]

epoch:   1 loss: 0.51251355
epoch:   6 loss: 0.38421459
epoch:  11 loss: 0.33365531
epoch:  16 loss: 0.27868673
epoch:  21 loss: 0.23237213
epoch:  26 loss: 0.22547626
epoch:  31 loss: 0.21850686
epoch:  36 loss: 0.18051512
epoch:  41 loss: 0.17885519
epoch:  46 loss: 0.14748415
epoch:  51 loss: 0.14732041
epoch:  56 loss: 0.14395057
epoch:  61 loss: 0.15388559
epoch:  66 loss: 0.14008746
epoch:  71 loss: 0.14489220
epoch:  76 loss: 0.11259133
epoch:  81 loss: 0.13803397
epoch:  86 loss: 0.13019930
epoch:  91 loss: 0.10053276
epoch:  96 loss: 0.09677521
epoch: 101 loss: 0.11732165
epoch: 106 loss: 0.15738848
epoch: 111 loss: 0.12271446
epoch: 116 loss: 0.11410480
epoch: 121 loss: 0.11098493
epoch: 126 loss: 0.13657261
epoch: 131 loss: 0.09615514
epoch: 136 loss: 0.09468974
epoch: 141 loss: 0.10673631
epoch: 146 loss: 0.07257059
epoch: 149 loss: 0.0078275073


In [10]:
!export CUBLAS_WORKSPACE_CONFIG=:16:8

In [1]:
seq

NameError: name 'seq' is not defined

In [20]:
model(torch.tensor(sample[0]).float().unsqueeze(0).cuda())

tensor([-0.5425], device='cuda:0', grad_fn=<SqueezeBackward1>)